In [6]:
import numpy as np
import json
from sklearn.feature_extraction import text

x = open('fedpapers_split.txt').read()
papers = json.loads(x)

papersH = papers[0] # papers by Hamilton 
papersM = papers[1] # papers by Madison
papersD = papers[2] # disputed papers

nH, nM, nD = len(papersH), len(papersM), len(papersD)

# This allows you to ignore certain common words in English
# You may want to experiment by choosing the second option or your own
# list of stop words, but be sure to keep 'HAMILTON' and 'MADISON' in
# this list at a minimum, as their names appear in the text of the papers
# and leaving them in could lead to unpredictable results
stop_words = text.ENGLISH_STOP_WORDS.union({'HAMILTON','MADISON'})
#stop_words = {'HAMILTON','MADISON'}

## Form bag of words model using words used at least 10 times
vectorizer = text.CountVectorizer(stop_words,min_df=10)
X = vectorizer.fit_transform(papersH+papersM+papersD).toarray()

# Uncomment this line to see the full list of words remaining after filtering out 
# stop words and words used less than min_df times
#vectorizer.vocabulary_

# Split word counts into separate matrices
XH, XM, XD = X[:nH,:], X[nH:nH+nM,:], X[nH+nM:,:]
print(papersH)
# Estimate probability of each word in vocabulary being used by Hamilton
#fH = XH/(XH+XM+XD)

# Estimate probability of each word in vocabulary being used by Madison
#fM = XM/(XH+XM+XD)

# Compute ratio of these probabilities
#fratio = fH/fM

# Compute prior probabilities 
#piH = ???
#piM = ???

#for xd in XD: # Iterate over disputed documents
    # Compute likelihood ratio for Naive Bayes model
    #LR = ???
    #if LR>0.5:
     #   print 'Hamilton'
    #else:
     #   print 'Madison'
    

['1\r\n\r\nGeneral Introduction\r\n\r\nFor the Independent Journal. Saturday, October 27, 1787\r\n\r\n\r\nHAMILTON\r\n\r\nTo the People of the State of New York:\r\n\r\nAFTER an unequivocal experience of the inefficacy of the subsisting\r\nfederal government, you are called upon to deliberate on a new\r\nConstitution for the United States of America. The subject speaks its\r\nown importance; comprehending in its consequences nothing less than the\r\nexistence of the UNION, the safety and welfare of the parts of which it\r\nis composed, the fate of an empire in many respects the most interesting\r\nin the world. It has been frequently remarked that it seems to have been\r\nreserved to the people of this country, by their conduct and example,\r\nto decide the important question, whether societies of men are really\r\ncapable or not of establishing good government from reflection and\r\nchoice, or whether they are forever destined to depend for their\r\npolitical constitutions on accident